# Loading Dataset

In [17]:
from glob import glob
from keras.utils import np_utils
from sklearn.datasets import load_files
import numpy as np


def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets


names = [item[20:-1] for item in sorted(glob("images/bones/train/*/"))]
train_files, train_targets = load_dataset('images/bones/train')
valid_files, valid_targets = load_dataset('images/bones/valid')
test_files, test_targets = load_dataset('images/bones/test')


# Loading Tensors

In [18]:
from keras.preprocessing import image
from tqdm import tqdm

def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


# Pre-processing Images

In [19]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255



100%|██████████| 24/24 [00:00<00:00, 189.19it/s]

# Creating CNN Model

In [20]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(133, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(133, activation='softmax'))

model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 28, 28, 64)        0         
__________

# Training The Model

In [24]:
from keras.callbacks import ModelCheckpoint

epochs = 10

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5',
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)


Train on 256 samples, validate on 24 samples
Epoch 1/10
256/256 [==============================] - 9s - loss: 0.8856 - acc: 0.6406 - val_loss: 1.1579 - val_acc: 0.5000
Epoch 2/10
256/256 [==============================] - 9s - loss: 0.8548 - acc: 0.6250 - val_loss: 1.2974 - val_acc: 0.5417
Epoch 3/10
256/256 [==============================] - 9s - loss: 0.7906 - acc: 0.6836 - val_loss: 1.5797 - val_acc: 0.4583
Epoch 4/10
256/256 [==============================] - 9s - loss: 0.5515 - acc: 0.7578 - val_loss: 1.5206 - val_acc: 0.5000
Epoch 5/10
256/256 [==============================] - 9s - loss: 0.4858 - acc: 0.7969 - val_loss: 1.9141 - val_acc: 0.4583
Epoch 6/10
256/256 [==============================] - 9s - loss: 0.4795 - acc: 0.8281 - val_loss: 1.6282 - val_acc: 0.5417
Epoch 7/10
256/256 [==============================] - 9s - loss: 0.4138 - acc: 0.8359 - val_loss: 1.6320 - val_acc: 0.6250
Epoch 8/10
256/256 [==============================] - 9s - loss: 0.4491 - acc: 0.8438 - val_lo

# Testing The Model

In [25]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Bones test accuracy: %.4f%%' % test_accuracy)


Bones test accuracy: 70.8333%


# Prediction Machine

In [27]:

def prediction_machine(img_path):
    prediction = np.argmax(model.predict(np.expand_dims(tensor, axis=0)))
    return prediction


# Bone Disease Diagnose

In [ ]:
    import matplotlib.pyplot as plt
    import cv2

    final_images = np.array(glob('images/final/*'))

    for img_path in final_images:
        p_img = cv2.imread(img_path)
        plt.imshow(p_img)
        plt.show()
        prediction = prediction_machine(img_path)
        print('Predicted Disease: {0}'.format(prediction))